In [1]:
!pip install torchvision
!pip install pyhessian
!pip install pytorchcv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/133.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 4.8 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/532.4 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 19.6 MB/s eta 0:00:00


In [2]:
import torch
from torchvision import datasets, transforms
from pyhessian import hessian # Hessian computation
from pytorchcv.model_provider import get_model as ptcv_get_model # model
import numpy as np
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import expm, expm_multiply
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [3]:

def getData(name='cifar10', train_bs=128, test_bs=1000):
    """
    Get the dataloader
    """
    if name == 'cifar10':
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465),
                                 (0.2023, 0.1994, 0.2010)),
        ])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465),
                                 (0.2023, 0.1994, 0.2010)),
        ])

        trainset = datasets.CIFAR10(root='../data',
                                    train=True,
                                    download=True,
                                    transform=transform_train)
        train_loader = torch.utils.data.DataLoader(trainset,
                                                   batch_size=train_bs,
                                                   shuffle=True)

        testset = datasets.CIFAR10(root='../data',
                                   train=False,
                                   download=False,
                                   transform=transform_test)
        test_loader = torch.utils.data.DataLoader(testset,
                                                  batch_size=test_bs,
                                                  shuffle=False)
    if name == 'cifar10_without_dataaugmentation':
        transform_train = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465),
                                 (0.2023, 0.1994, 0.2010)),
        ])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465),
                                 (0.2023, 0.1994, 0.2010)),
        ])

        trainset = datasets.CIFAR10(root='../data',
                                    train=True,
                                    download=True,
                                    transform=transform_train)
        train_loader = torch.utils.data.DataLoader(trainset,
                                                   batch_size=train_bs,
                                                   shuffle=True)

        testset = datasets.CIFAR10(root='../data',
                                   train=False,
                                   download=False,
                                   transform=transform_test)
        test_loader = torch.utils.data.DataLoader(testset,
                                                  batch_size=test_bs,
                                                  shuffle=False)

    return train_loader, test_loader

def hessian_vector_product(gradsH, params, v):
    """
    compute the hessian vector product of Hv, where
    gradsH is the gradient at the current point,
    params is the corresponding variables,
    v is the vector.
    """
    hv = torch.autograd.grad(gradsH,
                             params,
                             grad_outputs=v,
                             only_inputs=True,
                             retain_graph=True)
    return hv
def group_product(xs, ys):
    """
    the inner product of two lists of variables xs,ys
    :param xs:
    :param ys:
    :return:
    """
    return sum([torch.sum(x * y) for (x, y) in zip(xs, ys)])

In [4]:

def RademacherVector(n):
    """
        Input:
            n: num of components
        Output:
            vec: random vector from Rademacher distribution
    """

    v = [
        torch.randint_like(p, high=2)
        for p in n
    ]
    for v_i in v:
        v_i[v_i == 0] = -1

    return v

def Hutchinson(oracle, n, l: int):
    """
        Input:
            oracle: oracle for implicit matrix-vector multiplication with A
            n: size of the matrix A
            l: number of iteration to approximate the trace of the matrix A
        Output:
            approximation: approximation to the trace of A
    """

    assert l >= 0

    approximation = 0

    for iter in range(l):
        g = RademacherVector(n)
        approximation += group_product(oracle(g), g).cpu().item()

    return approximation / l

def SquaredFrobenius(oracle, n: int, l: int):
    """
        Input:
            oracle: oracle for implicit matrix-vector multiplication with A
            n: size of the matrix A
            l: number of iteration to approximate the frobenius norm of the matrix A
        Output:
            approximation: approximation to the frobenius norm of A
    """

    assert l >= 0

    approximation = 0
    
    for iter in range(l):
        g = oracle(RademacherVector(n))
        approximation += group_product(g, g).cpu().item()
    
    return approximation[0, 0] / l

def write_array(arr, file_name):
    f = open("/kaggle/working/" + file_name, 'w')
    for i in arr:
        f.write(str(i) + " ")
    f.write("\n")
    f.close()

In [5]:

def DeltaShiftRestart(prev, cur, iter, last_app, n: int, l0: int, l: int, q: int):
    """
        DeltaShift algorithm, but restart every q iterations
        Input:
            oracles: oracles for implicit matrix-vector multiplication with A1, ..., Am
            n: matrix size
            l0: number of iteration to approximate the trace of the matrix A1
            l: number of iteration to approximate the trace of the other matrixes
            q: number of iterations to restart
        Output:
            approximation: list of approximations to the trace of A1, ..., Am
    """

    if iter % q == 0:
        return Hutchinson(cur, n, l0)
    else:
        t = last_app
        for iter in range(l):
            g = RademacherVector(n)
            t += (group_product(cur(g), g).cpu().item() - group_product(prev(g), g).cpu().item()) / l
        return t


def DeltaShift(prev, cur, var, iter, last_app, n: int, l0: int, l: int):
    if iter == 0:
        approximation = 0
        N = 0

        for _ in range(l0):
            g = RademacherVector(n)
            z = cur(g)
            approximation += group_product(z, g).cpu().item()
            N += group_product(z, z).cpu().item()

        approximation /= l0
        N /= l0
        variance = 2 / l0 * N
        return approximation, variance
    else:
        approximation = 0
        z = []
        w = []
        g = []
        for j in range(l):
            current = RademacherVector(n)
            z.append(prev(current))
            w.append(cur(current))
            g.append(current)

        N = 0
        M = 0
        C = 0
        for j in range(l):
            N += group_product(z[j], z[j]).cpu().item() / l
            M += group_product(w[j], w[j]).cpu().item() / l
            C += group_product(w[j], z[j]).cpu().item() / l

        gamma = 1 - (2 * C) / (l * var + 2 * N)

        t = (1 - gamma) * last_app
        for j in range(l):
            t += (group_product(g[j], w[j]).cpu().item() - (1 - gamma) * group_product(g[j], z[j]).cpu().item()) / l

        variance = (1 - gamma)**2 * var + 2 / l * (M + (1 - gamma)**2 * N - 2 * (1 - gamma) * C)
        return t, variance

In [6]:
def GetTraceAndFrob(oracle, n, l: int):
    """
        Input:
            oracle: oracle for implicit matrix-vector multiplication with A
            n: size of the matrix A
            l: number of iteration to approximate the trace of the matrix A
        Output:
            approximation: approximation to the trace and to the squared frob-norm of A
    """

    assert l >= 0

    approximation_tr = 0
    approximation_fr = 0

    for iter in range(l):
        g = RademacherVector(n)
        t = oracle(g)
        approximation_tr += group_product(t, g).cpu().item()
        approximation_fr += group_product(t, t).cpu().item()

    return approximation_tr / l, approximation_fr / l

def DeltaShiftFrob(prev, cur, iter, last_app_e, last_app_h, n: int, l0: int, l: int):
    if iter == 0:
        approximation = 0

        for _ in range(l0):
            g = cur(RademacherVector(n))
            approximation += group_product(g, g).cpu().item()
        return approximation / l0, approximation / l0
    else:
        easy_way = last_app_e
        hard_way = last_app_h
        for _ in range(l // 2):
            g1 = prev(RademacherVector(n))
            g2 = cur(RademacherVector(n))
            t1 = group_product(g1, g2).cpu().item() - group_product(g1, g1).cpu().item()
            t2 = group_product(g2, g2).cpu().item() - 2 * group_product(g2, g1).cpu().item() + group_product(g1, g1).cpu().item()
            easy_way += 2 * t1 / (l // 2)
            hard_way += 2 * t1 / (l // 2)
            hard_way += t2 / (l // 2)
        
        return easy_way, hard_way

In [7]:

def CheckAlgorithms(oracles, n, correct_ans, title = None):
    """
        Generate graphic with relative error
        Input:
            oracles: list of oracle to compute matrix-vector multiplication
            n: matrix size
            correct_ans: list of the trace of the given matrixes
            title: title of the graphic
        Output:
            -
    """

    l0 = 100
    l = 50

    correct_ans = np.array(correct_ans)

    simpl_hutchinson = np.array(SimpleHutchinson(oracles, n, l))
    write_array(abs(correct_ans - simpl_hutchinson) / max(correct_ans), 'Hutchinson.txt')

    print("1")

    delta_shift = np.array(ParameterFreeDeltaShift(oracles, n, l0, l - 50 // (len(oracles) - 1)))
    write_array(abs(correct_ans - delta_shift) / max(correct_ans), 'Deltshift.txt')

    print("2")

    delta_shift_r = np.array(DeltaShiftRestart(oracles, n, l0, l - 50 // 19, 20))
    write_array(abs(correct_ans - delta_shift_r) / max(correct_ans), 'Deltshiftrest.txt')


In [8]:

# get the model
import datetime



def run_test(i):
    model = ptcv_get_model("resnet20_cifar10", pretrained=True)
    # change the model to eval mode to disable running stats upate
    model.eval()

    # create loss function
    criterion = torch.nn.CrossEntropyLoss()

    # get dataset
    train_loader, test_loader = getData()

    correct_ans = []
    prev = None
    varience = 0
    last_app = 0
    last_appr = 0

    last_ser = 0
    last_me = 0

    model = model.cuda()

    for ab, i in zip(train_loader, range(len(train_loader))):
        if i >= 8:
            break
        inputs, targets = ab
        hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=True)

        now = datetime.datetime.now()

        true_trace, true_frob = GetTraceAndFrob((lambda x: hessian_vector_product(hessian_comp.gradsH, hessian_comp.params, x)), hessian_comp.params, 1000)
        hutch_trace, hutch_frob = GetTraceAndFrob((lambda x: hessian_vector_product(hessian_comp.gradsH, hessian_comp.params, x)), hessian_comp.params, 50)
        delt_shift, var = DeltaShift((lambda x: hessian_vector_product(prev.gradsH, prev.params, x)),
                                (lambda x: hessian_vector_product(hessian_comp.gradsH, hessian_comp.params, x)),
                                varience, i, last_app, hessian_comp.params, 100, 50)
        delt_shift_r = DeltaShiftRestart((lambda x: hessian_vector_product(prev.gradsH, prev.params, x)),
                                (lambda x: hessian_vector_product(hessian_comp.gradsH, hessian_comp.params, x)),
                            i, last_appr, hessian_comp.params, 100, 50 - (50 // 19), 20)
        sereja, me = DeltaShiftFrob((lambda x: hessian_vector_product(prev.gradsH, prev.params, x)),
                                (lambda x: hessian_vector_product(hessian_comp.gradsH, hessian_comp.params, x)),
                            i, last_ser, last_me, hessian_comp.params, 100, 50)
        prev = hessian_comp
        last_app = delt_shift
        last_appr = delt_shift_r
        varience = var
        last_ser = sereja
        last_me = me

        correct_ans.append((true_trace, hutch_trace, delt_shift, delt_shift_r, true_frob, hutch_frob, sereja, me))
        print(true_trace, hutch_trace, delt_shift, delt_shift_r, true_frob, hutch_frob, sereja, me, datetime.datetime.now() - now)
        write_array(correct_ans, f"correct_{i}.txt")

    print(correct_ans)
    write_array(correct_ans, f"correct_{i}.txt")

for i in range(12):
    run_test(i)


  0%|          | 0/170498071 [00:00<?, ?it/s]


  0%|          | 65536/170498071 [00:00<04:33, 622284.29it/s]


  0%|          | 229376/170498071 [00:00<02:27, 1153842.01it/s]


  1%|          | 917504/170498071 [00:00<00:47, 3539221.17it/s]


  2%|▏         | 3702784/170498071 [00:00<00:13, 12298145.77it/s]

...

17322.393390625 16999.28177734375 17382.15294800708 17976.790008138025 8303054.08 8097037.695 -135837793.8354883 6721376.383310551 0:40:05.839361


18139.638240234373 18610.8790625 17953.29517161014 18613.96914632161 9177168.096375 9718710.06 -157191702.8867383 6390076.543310554 0:41:42.278814


18415.77846875 18146.08345703125 18719.453085653873 19301.252532552076 9435503.6745 8931019.3125 -179998006.80673835 5354562.103310555 0:43:18.052718


19126.382688476562 19288.21837890625 19285.57176379818 19779.77338623048 10179226.407375 10612930.84 -198410850.83423838 4757339.543310553 0:44:53.997162


20232.33291699219 20427.238359375 20126.315877854053 20593.316578776048 11374036.3175 11521245.645 -218414595.59455097 7002635.363310553 0:46:30.660742
